IBIOGRAM Build datasets for map service

This first requires that the files be converted to parquet


In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
#import duckdb
import csv
import os
import sys
import logging  
from keplergl import KeplerGl
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(asctime)s %(message)s")
logger=logging.getLogger(__name__)
logger.setLevel(level=logging.DEBUG)


In [2]:
base_dir = os.getcwd()
logger.info("base dir: {}".format(base_dir))

data_dir = os.path.join(base_dir, "../../data/ibiogram/latest/")
files = {
    "demo": {"file":"decs-1290-demo.txt","parse_dates":[], "dtype": {}},
    
"problems":{"file":"decs-1290-problems.txt","parse_dates":["NOTED_DATE"], "dtype":{}},
"visits":{"file":"decs-1290-visits.txt",
          "parse_dates":["HOSP_ADMSN_TIME","HOSP_DISCH_TIME"],
          "dtype":{
              "PAT_ENC_CSN_DI": 'string'
          }},
"sensitivity":{"file":"decs-1290-sensitivity.txt","parse_dates":["RESULT_DATE"], "dtype":{}},
}
output_dir = os.path.join(base_dir, "../../data/ibiogram/output/")

2024-05-07 12:31:52,879 base dir: /Users/valentin/development/dev_resilient/resilient_workflows/sheild/notebooks


Load Zip shapefiles

In [3]:
zip_codes_gdf = gpd.read_file(os.path.join(base_dir, "../../data/geographic/sd_county/sd_county_zip_codes.gpkg"))
zip_codes_gdf = zip_codes_gdf.set_crs(epsg=4326)
zip_codes_gdf['LON'] = zip_codes_gdf.centroid.x
zip_codes_gdf['LAT'] = zip_codes_gdf.centroid.y 
zip_codes_gdf['POINT']= zip_codes_gdf.centroid
zip_points_gdf=zip_codes_gdf.set_geometry("POINT",drop=True, inplace=False) 
 
zip_codes_gdf

/var/folders/t2/t39bprkn16dg9nr4v6c18v0w0000gn/T/ipykernel_68908/3105166035.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  zip_codes_gdf['LON'] = zip_codes_gdf.centroid.x
/var/folders/t2/t39bprkn16dg9nr4v6c18v0w0000gn/T/ipykernel_68908/3105166035.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  zip_codes_gdf['LAT'] = zip_codes_gdf.centroid.y
/var/folders/t2/t39bprkn16dg9nr4v6c18v0w0000gn/T/ipykernel_68908/3105166035.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  zip_codes_gdf['POINT']= zip_codes_gdf.centroid


,OBJECTID,ZIP_CODE,geometry,LON,LAT,POINT
0,1,91901,"MULTIPOLYGON (((-116.60749 32.79858, -116.6101...",-116.714872,32.807958,POINT (-116.71487 32.80796)
1,2,91902,"MULTIPOLYGON (((-116.96415 32.67248, -116.9643...",-117.005161,32.674016,POINT (-117.00516 32.67402)
2,3,91905,"MULTIPOLYGON (((-116.20848 32.66050, -116.2086...",-116.301852,32.726552,POINT (-116.30185 32.72655)
3,4,91906,"MULTIPOLYGON (((-116.35730 32.70081, -116.3565...",-116.516699,32.696103,POINT (-116.51670 32.69610)
4,5,91910,"MULTIPOLYGON (((-117.07281 32.65629, -117.0727...",-117.055637,32.637943,POINT (-117.05564 32.63794)
...,...,...,...,...,...,...
117,118,92590,"MULTIPOLYGON (((-117.25103 33.45183, -117.2513...",-117.274805,33.459686,POINT (-117.27481 33.45969)
118,119,92592,"MULTIPOLYGON (((-116.87247 33.42673, -116.8759...",-117.008187,33.429030,POINT (-117.00819 33.42903)
119,120,92672,"MULTIPOLYGON (((-117.57400 33.45341, -117.5736...",-117.536888,33.385831,POINT (-117.53689 33.38583)
120,121,92673,"MULTIPOLYGON (((-117.57400 33.45341, -117.5743...",-117.577576,33.452593,POINT (-117.57758 33.45259)


ZIP Codes US

In [4]:
#ountry code      : iso country code, 2 characters
#postal code       : varchar(20)
#place name        : varchar(180)
#admin name1       : 1. order subdivision (state) varchar(100)
#admin code1       : 1. order subdivision (state) varchar(20)
#admin name2       : 2. order subdivision (county/province) varchar(100)
#admin code2       : 2. order subdivision (county/province) varchar(20)
#admin name3       : 3. order subdivision (community) varchar(100)
#admin code3       : 3. order subdivision (community) varchar(20)
#latitude          : estimated latitude (wgs84)
#longitude         : estimated longitude (wgs84)
#accuracy          : accuracy of lat/lng from 1=estimated, 4=geonameid, 6=centroid of addresses or shape
geo_data_dir = os.path.join(base_dir, "../../data/geographic/")
FILE_NAME = f"{geo_data_dir}/us_zip_point/US.txt"
FILE_HEADER = ['Country', 'ZIP_CODE', 'Place_name', 
               'ADMIN_NAME_1', 'ADMIN_CODE_1',
               'ADMIN_NAME_2', 'ADMIN_CODE_2',
               'ADMIN_NAME_3', 'ADMIN_CODE_3',
               'latitude', 'longitude',
              'accuracy' ]
USE_COLS = ['ZIP_CODE', 'latitude', 'longitude']
DTYPE= {'ZIP_CODE':str,}
uszip_df = pd.read_csv(
    FILE_NAME, delimiter="\t", header=None,
    names=FILE_HEADER, usecols=USE_COLS,
dtype=DTYPE)

uszip_gdf = gpd.GeoDataFrame(uszip_df, 
    geometry = gpd.points_from_xy(uszip_df['longitude'], uszip_df['latitude']), 
    crs = 'EPSG:4326')

uszip_gdf.head()

,ZIP_CODE,latitude,longitude,geometry
0,99553,54.1430,-165.7854,POINT (-165.78540 54.14300)
1,99571,55.1858,-162.7211,POINT (-162.72110 55.18580)
2,99583,54.8542,-163.4113,POINT (-163.41130 54.85420)
3,99612,55.0628,-162.3056,POINT (-162.30560 55.06280)
4,99661,55.3192,-160.4914,POINT (-160.49140 55.31920)


Load Parquet
In the long term, these will be on an s3 store, and may be stored in Apache Arrow to allow for subsetting

In [5]:
demo_df = pd.read_parquet(os.path.join(output_dir, "parquet","demo.parquet"))
demo_df.dropna(subset=['ZIP'], inplace=True)
demo_df = demo_df[ (demo_df['ZIP'].str.startswith('9') )]
demo_df['ZIP_CODE']=demo_df['ZIP']
sens_df= pd.read_parquet(os.path.join(output_dir,"parquet", "sensitivity.parquet"))


In [6]:
organisims= sens_df["ORGANISM"].unique()

def SpitSpeciesGenus(organismRow):
    terms = organismRow.split(" ", maxsplit=1)
    if (len(terms) ==1):
        return terms[0], ""
    else:
        return terms[0], terms[1]
    
sens_df["genus"],sens_df["species"] = zip(*sens_df["ORGANISM"].apply(SpitSpeciesGenus))
sens_df
organisims_list = sens_df[["ORGANISM","genus","species" , "SUSCEPT"]].groupby([  "genus","species", "ORGANISM"
                                                                    ]).agg(count=("ORGANISM", "count"))
organisims_list.to_csv(os.path.join(output_dir,"pathogen",f"bacteria_names_count.csv" ))

genus_list = (sens_df[["genus", "SUSCEPT"]].
              groupby([  "genus", "SUSCEPT" ], as_index=False).agg(count=("SUSCEPT", "count")))
genus_list.sort_values("count", ascending=False).to_csv(os.path.join(output_dir,"pathogen",f"bacteria_genus_suspect_count.csv" ))
genus_list = sens_df[["genus", "SUSCEPT"]].groupby([  "genus" ], as_index=False).agg(count=("SUSCEPT", "count"))
genus_list.sort_values("count", ascending=False).to_csv(os.path.join(output_dir,"pathogen",f"bacteria_genus.csv" ))

organisims= sens_df[["ORGANISM","genus","species", "SUSCEPT" ]].groupby(["genus","species","SUSCEPT" ]).agg(count=("ORGANISM", "count"))

organisims

count
genus         species        SUSCEPT            
ABIOTROPHIA   DEFECTIVA      Intermediate      4
                             Resistant         6
                             Susceptible      41
ACHROMOBACTER                Intermediate     16
                             Resistant       143
...                                          ...
YERSINIA      ENTEROCOLITICA Resistant        46
                             Susceptible     157
              FREDERIKSENII  Resistant         1
                             Susceptible       5
              ROHDEI         Susceptible      14

[1387 rows x 1 columns]

In [7]:

mrn_zip_gby = demo_df[["MRN_DI","ZIP_CODE","PAT_SEX","PAT_RACE"]].groupby(by=["MRN_DI","ZIP_CODE",'PAT_SEX',"PAT_RACE",], as_index=False).count()
mrn_zip_df = mrn_zip_gby.reset_index()
mrn_zip_df.head(50)

,index,MRN_DI,ZIP_CODE,PAT_SEX,PAT_RACE
0,0,90000001,92131,M,White
1,1,90000002,92101,F,White
2,2,90000003,92071,M,White
3,3,90000004,92122,F,White
4,4,90000005,92129,M,Asian
5,5,90000006,92123,M,White
6,6,90000007,91950,M,White
7,7,90000008,92596,F,White
8,8,90000009,91914,M,Other Race or Mixed Race
9,9,90000010,91978,F,White


In [8]:
sens_org_gby = sens_df[["MRN_DI","genus", "species", "ORGANISM","RESULT_DATE","SUSCEPT","ANTIBIOTIC" ]].groupby(by=["genus", "species","ORGANISM","ANTIBIOTIC","SUSCEPT","MRN_DI","RESULT_DATE" ]).count()
sens_org_df= sens_org_gby.reset_index()
sens_org_df.head(50)

,genus,species,ORGANISM,ANTIBIOTIC,SUSCEPT,MRN_DI,RESULT_DATE
0,ABIOTROPHIA,DEFECTIVA,ABIOTROPHIA DEFECTIVA,Ampicillin,Resistant,90225035,2021-02-02
1,ABIOTROPHIA,DEFECTIVA,ABIOTROPHIA DEFECTIVA,Ampicillin,Susceptible,90028589,2015-05-29
2,ABIOTROPHIA,DEFECTIVA,ABIOTROPHIA DEFECTIVA,Ceftriaxone,Resistant,90028589,2015-05-29
3,ABIOTROPHIA,DEFECTIVA,ABIOTROPHIA DEFECTIVA,Ceftriaxone,Resistant,90259507,2015-08-10
4,ABIOTROPHIA,DEFECTIVA,ABIOTROPHIA DEFECTIVA,Ceftriaxone,Susceptible,90000054,2015-10-02
5,ABIOTROPHIA,DEFECTIVA,ABIOTROPHIA DEFECTIVA,Ceftriaxone,Susceptible,90011788,2018-12-18
6,ABIOTROPHIA,DEFECTIVA,ABIOTROPHIA DEFECTIVA,Ceftriaxone,Susceptible,90036705,2020-10-23
7,ABIOTROPHIA,DEFECTIVA,ABIOTROPHIA DEFECTIVA,Ceftriaxone,Susceptible,90038222,2020-04-08
8,ABIOTROPHIA,DEFECTIVA,ABIOTROPHIA DEFECTIVA,Ceftriaxone,Susceptible,90073728,2022-01-31
9,ABIOTROPHIA,DEFECTIVA,ABIOTROPHIA DEFECTIVA,Ceftriaxone,Susceptible,90080549,2019-06-28


In [9]:
join_df = pd.merge( mrn_zip_df, sens_org_df, on="MRN_DI" )
join_df.drop(["MRN_DI",'index'], axis=1, inplace=True)
join_df.head(50)
join_df.to_csv(os.path.join(output_dir,"merged", "bacteria_suspect_zip.csv"), index=False)

In [12]:

join_short_gby = join_df.groupby(by=["ZIP_CODE","RESULT_DATE","genus" 
                                            ], as_index=False).agg(count=("ORGANISM", "count"))

join_short_gby.to_csv(os.path.join(output_dir,"merged", "bacteria_genus_date_zip.csv"), index=False)
join_short_gby.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93803 entries, 0 to 93802
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ZIP_CODE     93803 non-null  object        
 1   RESULT_DATE  93803 non-null  datetime64[ns]
 2   genus        93803 non-null  object        
 3   count        93803 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 2.9+ MB


Dataframe for generating Zip Code layers

In [14]:
bacteria_genus_date_zip=pd.merge( uszip_gdf, join_short_gby, on="ZIP_CODE" )
bacteria_genus_date_zip.head()
bacteria_genus_date_zip.to_file(os.path.join(output_dir,"merged", "bacteria_genus_date_zip.geojson" ) , driver='GeoJSON')

In [11]:
join_short_gby = join_df.groupby(by=["ZIP_CODE"
                                           ,"RESULT_DATE", ], as_index=False).agg(count=("genus", "count"))
join_short_gby.to_csv(os.path.join(output_dir,"merged", "bacteria_date_count_zip.csv"), index=False)
join_short_gby.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64174 entries, 0 to 64173
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ZIP_CODE     64174 non-null  object        
 1   RESULT_DATE  64174 non-null  datetime64[ns]
 2   count        64174 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 1.5+ MB


In [13]:
join_short_gby = join_df.groupby(by=["ZIP_CODE"
                                            ], as_index=False).agg(count=("genus", "count"))

join_short_gby.to_csv(os.path.join(output_dir,"merged", "bacteria_zip_count.csv"), index=False)
join_short_gby.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1828 entries, 0 to 1827
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ZIP_CODE  1828 non-null   object
 1   count     1828 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 28.7+ KB


In [14]:

join_short_gby = join_df.groupby(by=["ZIP_CODE","genus" 
                                            ], as_index=False).agg(count=("ORGANISM", "count"))

join_short_gby.to_csv(os.path.join(output_dir,"merged", "bacteria_genus_zip.csv"), index=False)
join_short_gby.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8591 entries, 0 to 8590
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ZIP_CODE  8591 non-null   object
 1   genus     8591 non-null   object
 2   count     8591 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 201.5+ KB


In [15]:
sens_org_antibotic_gby = join_df[["ZIP_CODE","genus", "species", "SUSCEPT","ANTIBIOTIC" ]].groupby(by=["ZIP_CODE", "genus", "species","ANTIBIOTIC","SUSCEPT"], as_index=False).agg(count=("SUSCEPT", "count"))
 
sens_org_antibotic_gby.head(50)

,ZIP_CODE,genus,species,ANTIBIOTIC,SUSCEPT,count
0,90001,ESBL,ESCHERICHIA COLI,Amikacin,Susceptible,1
1,90001,ESBL,ESCHERICHIA COLI,Ampicillin,Resistant,1
2,90001,ESBL,ESCHERICHIA COLI,Ampicillin/Sulbactam,Resistant,1
3,90001,ESBL,ESCHERICHIA COLI,Cefazolin,Resistant,1
4,90001,ESBL,ESCHERICHIA COLI,Cefepime,Resistant,1
5,90001,ESBL,ESCHERICHIA COLI,Cefoxitin,Resistant,1
6,90001,ESBL,ESCHERICHIA COLI,Ceftazidime,Resistant,1
7,90001,ESBL,ESCHERICHIA COLI,Ceftriaxone,Resistant,1
8,90001,ESBL,ESCHERICHIA COLI,Ciprofloxacin,Susceptible,1
9,90001,ESBL,ESCHERICHIA COLI,Ertapenem,Susceptible,1


BY GENUS

In [16]:
sens_genus_antibotic_gby = join_df[["ZIP_CODE","genus", ]].groupby(by=["ZIP_CODE", "genus"], as_index=False).agg(count=("genus", "count"))
sens_genus_antibotic_gby.info()
sens_genus_antibotic_gby.head(100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8591 entries, 0 to 8590
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ZIP_CODE  8591 non-null   object
 1   genus     8591 non-null   object
 2   count     8591 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 201.5+ KB


,ZIP_CODE,genus,count
0,90001,ESBL,16
1,90002,CANDIDA,2
2,90002,ENTEROCOCCUS,6
3,90002,ESCHERICHIA,48
4,90002,KLEBSIELLA,72
...,...,...,...
95,90094,KLEBSIELLA,15
96,90094,PROTEUS,28
97,90201,ESCHERICHIA,32
98,90211,ENTEROCOCCUS,6


In [18]:
zip_genus_max_gpd=pd.merge( uszip_gdf, sens_genus_antibotic_gby, on="ZIP_CODE" )
zip_genus_max_gpd.to_file(os.path.join(output_dir,"merged","bacteria_max_zip.geojson" ) , driver='GeoJSON')
zip_genus_max_gpd.to_csv(os.path.join(output_dir,"merged","bacteria_max_zip.csv"))

In [17]:
sens_genus_max = sens_genus_antibotic_gby.groupby(["ZIP_CODE"],as_index=False).max()
sens_genus_max.head(100)  

,ZIP_CODE,genus,count
0,90001,ESBL,16
1,90002,SERRATIA,72
2,90003,STAPHYLOCOCCUS,11
3,90004,METHICILLIN,12
4,90006,PSEUDOMONAS,18
...,...,...,...
95,90704,PROTEUS,15
96,90706,KLEBSIELLA,80
97,90710,STREPTOCOCCUS,32
98,90712,ESCHERICHIA,16


In [15]:
    
zip_genus_max=pd.merge( uszip_gdf, sens_genus_max, on="ZIP_CODE" )
zip_genus_max.to_file(os.path.join(output_dir,"genus","bacteria_genus_max_zip.geojson" ) , driver='GeoJSON')
zip_genus_max.to_csv(os.path.join(output_dir,"merged","bacteria_genus_max_zip.csv"))

NameError: name 'sens_genus_max' is not defined

In [54]:
def generateGenusZip(genus,dataframe, path="genus"):
    logger.info(f"Generating genus {genus} to {path}")
    filter_df = dataframe[dataframe["genus"] == genus]
    zip_genus=pd.merge( zip_genus_max, filter_df, on="ZIP_CODE" )
   # zip_genus.drop("MRN_DI")
    if len(zip_genus) == 0:
        return
    zip_genus.to_file(os.path.join(output_dir,path,f"bacteria_{genus}_zip.geojson" ) , driver='GeoJSON')
    zip_genus.to_csv(os.path.join(output_dir,path,f"bacteria_{genus}_zip.csv" ))
    zip_genus.head(20)

top20=genus_list.sort_values(by=["count"], ascending=False, inplace=False,).head(20)['genus']
for g in top20:
    generateGenusZip(g,join_df, path="top20")
    
genus = join_df["genus"].unique()
for g in genus:
    generateGenusZip(g,join_df)



2024-05-04 20:14:26,096 Generating genus ESCHERICHIA to top20
2024-05-04 20:14:48,899 Generating genus KLEBSIELLA to top20
2024-05-04 20:14:59,120 Generating genus STAPHYLOCOCCUS to top20
2024-05-04 20:15:07,927 Generating genus ESBL to top20
2024-05-04 20:15:15,839 Generating genus PSEUDOMONAS to top20
2024-05-04 20:15:22,541 Generating genus PROTEUS to top20
2024-05-04 20:15:28,246 Generating genus METHICILLIN to top20
2024-05-04 20:15:33,548 Generating genus ENTEROBACTER to top20
2024-05-04 20:15:37,513 Generating genus ENTEROCOCCUS to top20
2024-05-04 20:15:40,306 Generating genus CITROBACTER to top20
2024-05-04 20:15:42,487 Generating genus SERRATIA to top20
2024-05-04 20:15:44,096 Generating genus MORGANELLA to top20
2024-05-04 20:15:45,390 Generating genus EXTENDED to top20
2024-05-04 20:15:46,531 Generating genus VANCOMYCIN to top20
2024-05-04 20:15:47,366 Generating genus PROVIDENCIA to top20
2024-05-04 20:15:48,184 Generating genus STREPTOCOCCUS to top20
2024-05-04 20:15:49,0

In [47]:
zip_genus_max

,ZIP_CODE,latitude,longitude,geometry,genus,count
0,99692,53.8898,-166.5422,POINT (-166.54220 53.88980),VANCOMYCIN,7
1,99502,61.1661,-149.9600,POINT (-149.96000 61.16610),ESCHERICHIA,16
2,99503,61.1900,-149.8938,POINT (-149.89380 61.19000),STENOTROPHOMONAS,30
3,99507,61.1535,-149.8289,POINT (-149.82890 61.15350),STAPHYLOCOCCUS,19
4,99516,61.1054,-149.7800,POINT (-149.78000 61.10540),ESCHERICHIA,30
...,...,...,...,...,...,...
984,99223,47.6156,-117.3622,POINT (-117.36220 47.61560),ENTEROCOCCUS,6
985,98516,47.1126,-122.7794,POINT (-122.77940 47.11260),SERRATIA,47
986,98226,48.7974,-122.4448,POINT (-122.44480 48.79740),MYCOBACTERIUM,4
987,98264,48.9372,-122.4592,POINT (-122.45920 48.93720),ESBL,15


In [36]:
zip_focus= zip_genus_max.dissolve(by="ZIP_CODE")
zip_focus.head(50)

,geometry,latitude,longitude,genus,count
ZIP_CODE,,,,,
90001,POINT (-118.24790 33.97310),33.9731,-118.2479,ESBL,16
90002,POINT (-118.24620 33.94970),33.9497,-118.2462,SERRATIA,72
90003,POINT (-118.27270 33.96530),33.9653,-118.2727,STAPHYLOCOCCUS,11
90004,POINT (-118.30290 34.07620),34.0762,-118.3029,METHICILLIN,12
90006,POINT (-118.29170 34.04930),34.0493,-118.2917,PSEUDOMONAS,18
90008,POINT (-118.34110 34.01160),34.0116,-118.3411,KLEBSIELLA,16
90010,POINT (-118.30270 34.06060),34.0606,-118.3027,ESCHERICHIA,150
90011,POINT (-118.25820 34.00790),34.0079,-118.2582,STREPTOCOCCUS,2
90013,POINT (-118.24340 34.04480),34.0448,-118.2434,ENTEROBACTER,15


In [37]:
import fiona; fiona.supported_drivers

zip_focus.to_parquet(os.path.join(output_dir,"zip_focus.parquet" ))
zip_focus.to_file(os.path.join(output_dir,"zip_focus.geojson" ) , driver='GeoJSON')
zip_focus.to_file(os.path.join(output_dir,"zip_focus.flatgeobuf" ) , driver='FlatGeobuf')